# Embedding Projector

[source](https://stackoverflow.com/questions/57014236/how-to-use-the-embedding-projector-in-tensorflow-2-0)

In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

import os
import tensorflow as tf
from tensorboard.plugins import projector
import tensorboard
import io

In [2]:
# log_dir='../../artifact/projector/'

PATH_INTERIM = '../../data/interim/' 
PATH_MODEL = '../../model/'
# PATH_LogDir = '../../model/projectors/'
PATH_LogDir = '../../model/multi_projectors/'

In [3]:
tensorboard.__version__

## '2.5.0'

'2.5.0'

In [4]:
tf.__version__

'2.3.0'

## Preparation

In [5]:
df_basket = pd.read_csv(PATH_INTERIM + 'all_basket2.csv')
all_items = list(set((','.join(df_basket['norms'])).split(',')))
all_items.sort()

In [6]:
# all_items

## Generate The Vectors

In [7]:
embs = ["w2v_50_3_1.model","w2v_100_3_1.model","w2v_50_3_2.model","w2v_100_3_2.model"]

all_v_list = []
all_metadata = []
len(all_items)

269

In [8]:
# embs_temp = embs[0]
for embs_temp in embs :
    w2v_temp = Word2Vec.load(PATH_MODEL+embs_temp)
    all_v = []
    meta_items = []
    for word in all_items :
        try :
            vec = w2v_temp.wv[word]
            all_v.append(vec)
            meta_items.append(word) 
           
           
        except:
            pass
        
    all_v = np.array(all_v)

    all_v_list.append(all_v)
    all_metadata.append(meta_items)

In [9]:
all_v_list[0].shape,all_v_list[1].shape

((211, 150), (211, 150))

In [10]:
len(all_metadata[0]),len(all_metadata[1])

(211, 211)

## Create Meta

In [11]:
# df_meta2 = df_meta[['#','Name']]
# df_meta2[:100].to_csv(log_dir + 'meta_mini.tsv', sep="\t")
# df[:100].to_csv(log_dir + 'vecs_mini.tsv', sep="\t", header=False)

In [12]:
all_feats = []
for i in range(len(embs)):
    
    metadata_used = all_metadata[i]
    vector_used = all_v_list[i]
    
    metadata_file = 'meta'+str(i)+'.tsv'
#     vector_file = 'embed'+str(i)+'.ckpt'

    ## Metadata
    # pd.DataFrame(metadata_used).to_csv(PATH_LogDir + metadata_file, sep="\t", index = False)
    out_m = io.open(PATH_LogDir+metadata_file, 'w', encoding='utf-8')
    for word in metadata_used:
        out_m.write(word + "\n")
    out_m.close()
    print(metadata_file+':',len(metadata_used))

    ## Vector
    we = tf.Variable(vector_used, trainable=False,name=('embedM_'+str(i)))
    # we = tf.Variable(vector_used, trainable=False)
    
    # vector_file = we.name[:-2]+'.ckpt'
    # checkpoint = tf.train.Checkpoint(embedding=we)
    # checkpoint.save(PATH_LogDir + vector_file)
    all_feats.append(we)
    
saver = tf.compat.v1.train.Saver(all_feats)  # Must pass list or dict
saver.save(sess=None, global_step=0, save_path=PATH_LogDir+'all_embeddings.cpkt')

meta0.tsv: 211
meta1.tsv: 211
meta2.tsv: 259
meta3.tsv: 259


'../../model/multi_projectors/all_embeddings.cpkt-0'

In [13]:
reader = tf.train.load_checkpoint(PATH_LogDir)
map = reader.get_variable_to_shape_map()
key_to_use = ""
for key in map:
    if "embeddings" in key:
        print(key)

In [14]:
reader.get_variable_to_shape_map()

{'embedM_0': [211, 150],
 'embedM_1': [211, 150],
 'embedM_2': [259, 150],
 'embedM_3': [259, 150]}

In [15]:
# tf.summary.create_file_writer

In [16]:
k = 0
config = projector.ProjectorConfig()
for wv0 in all_feats:
    metadata_file = 'meta'+str(k)+'.tsv'
    
    # Set up config.
    
    embedding = config.embeddings.add()
    # The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
    # embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
    # embedding.tensor_name = "embedding_15k_v7.cpkt"
    embedding.tensor_name = wv0.name[:-2]
    # embedding.tensor_path = wv0.name[:-2]
    embedding.metadata_path = metadata_file
    k += 1
projector.visualize_embeddings(PATH_LogDir, config)

In [17]:
config

embeddings {
  tensor_name: "embedM_0"
  metadata_path: "meta0.tsv"
}
embeddings {
  tensor_name: "embedM_1"
  metadata_path: "meta1.tsv"
}
embeddings {
  tensor_name: "embedM_2"
  metadata_path: "meta2.tsv"
}
embeddings {
  tensor_name: "embedM_3"
  metadata_path: "meta3.tsv"
}

In [18]:
# %load_ext tensorboard

In [19]:
# %tensorboard --logdir model/multi_projectors
# %tensorboard --logdir ../../model/projectors
# !kill 17424

## References

[word2vec-embeddings-in-tensorboard](https://stackoverflow.com/questions/50492676/visualize-gensim-word2vec-embeddings-in-tensorboard-projector)

[projector_plugin](https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin)

In [20]:
tensorboard.__version__

'2.5.0'

In [13]:
# !pip install tensorboard==2.3.0

In [16]:
# https://stackoverflow.com/questions/45020971/visualizing-multiple-embedding-with-tensorflow 